In [ ]:
import pandas as pd
import numpy as np

## 1. 데이터 로드 및 준비

In [ ]:
# 전처리된 리뷰 데이터 로드
df_reviews = pd.read_pickle('./reviews.pickle')

# 모델 정보 추출을 위한 데이터프레임 복사 및 중복 제거
df_models = df_reviews[['브랜드', '제품명']].copy()
df_models.drop_duplicates(inplace=True)
df_models.reset_index(drop=True, inplace=True)

## 2. 모델 정보 추출 및 정제

In [ ]:
# 정규표현식을 사용하여 용량 정보 추출
pattern = r'(\d+(?:GB|TB))'
df_models['용량'] = df_models['제품명'].str.extract(pattern)

# 제품명에서 용량 정보를 제거하여 기본 모델명 생성
df_models['모델'] = df_models['제품명']
storage_list = df_models['용량'].dropna().unique().tolist()
for storage_name in storage_list:
    df_models['모델'] = df_models['모델'].str.replace(storage_name, '', regex=False)

In [ ]:
# 모델명에서 부가 정보(5G, 플러스 등) 제거
model_replace_pattern = r'5G|플러스|울트라|프로|미니|맥스|FE'
df_models['모델'] = (df_models['모델']
                   .str.replace(model_replace_pattern, '', regex=True)
                   .str.replace('A52S', 'A52') # A52S -> A52 통일
                   .str.strip())


In [ ]:
# 용량이 제거된 제품명 컬럼 생성
df_models['제품명(용량x)'] = df_models['제품명']
for storage_name in storage_list:
    df_models['제품명(용량x)'] = df_models['제품명(용량x)'].str.replace(storage_name, '', regex=False)

In [ ]:
# 모델명을 기준으로 시리즈 분류
df_models['시리즈'] = np.nan
df_models.loc[df_models['모델'].str.contains('갤럭시A', na=False), '시리즈'] = 'A 시리즈'
df_models.loc[df_models['모델'].str.contains('갤럭시S', na=False), '시리즈'] = 'S 시리즈'
df_models.loc[df_models['모델'].str.contains('노트', na=False), '시리즈'] = '노트 시리즈'
df_models.loc[df_models['모델'].str.contains('폴드|플립', na=False, regex=True), '시리즈'] = '폴더블'

# 아이폰의 경우, '시리즈'에 '모델'명을 그대로 사용
iphone_mask = df_models['모델'].str.contains('아이폰', na=False)
df_models.loc[iphone_mask, '시리즈'] = df_models.loc[iphone_mask, '모델']

## 3. 최종 데이터셋 저장

In [ ]:
# 최종 컬럼 선택 및 정렬
final_cols = ['제품명', '브랜드', '시리즈', '모델', '용량', '제품명(용량x)']
df_models = df_models[final_cols]
df_models.sort_values(by=['브랜드', '시리즈', '모델', '용량'], inplace=True)
df_models.reset_index(drop=True, inplace=True)

# 정제된 모델 데이터프레임 저장
df_models.to_pickle('./models.pickle')